#### Gonna try something epic here

My idea is to create a function/script/ability to overlay a sankey diagram over a bus route to represent the ons/offs and current passenger load, all while being displayed on the map. Hopefully would include hover/tooltip capabilities as well. IDK really how I will start but heres to trying

I will try initially to use the E line to do this. I already have the segment data which will be helpful but the main reason I am using the eline data is that it is mostly straight!! This will help as I am developing for a few reasons.

##### Relevant resources 

https://python-visualization.github.io/folium   ---> folium docs

In [1]:
### Gonna start with the imports. Nothing weird don't freak out.
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely

import folium
import json
import branca.colormap as cm

pd.set_option('display.max_columns', None) ### This line makes all the columns display, rather than ellipses shorten

In [2]:
### import the e line data with segments

kcm_segment_data = "C:/Users/aaron/CoreGIS_Work/CoreGIS Work Sync/COREGIS_work/RapidRide E/Data/Source/Transit_Routes_for_King_County_Metro___transitroute_line"
kcm_point_data = "C:/Users/aaron/CoreGIS_Work/CoreGIS Work Sync/COREGIS_work/RapidRide E/Data/Source/Transit_Stops_for_King_County_Metro___transitstop_point"

all_kcm_segment_data = gpd.read_file(kcm_segment_data)
all_kcm_point_data = gpd.read_file(kcm_point_data)


In [3]:
### gonnna fabricate some data about the ons/offs/avg departing load. This isn't about reflecting reality at the current moment
## Stop list, south to north --> [7690, 7710, 7730, 7750, 7770, 7790, 7810, 7850]

##completely made up
## starting_stop_num: [boardings, alightings]

#type = dict
ridership_data = {
    7690: [12, 0],
    7710: [14, 6],
    7730: [13, 2],
    7750: [5, 20],
    7770: [8, 6],
    7790: [10, 14],
    7810: [3, 6],
    7850: [2, 7],
    7880: [1, 6]
}



In [4]:
### compiling a list of stops that I will use for this experiment. Mostly in a straight line and not too many of them.
#e_line_northbound_stop_list = [7690, 7710, 7730, 7750, 7770, 7790, 7810, 7850, 7880]
e_line_northbound_stop_list = list(ridership_data.keys())


e_line_northbound_stops = all_kcm_point_data[all_kcm_point_data['STOP_ID'].isin(e_line_northbound_stop_list)]
e_line_northbound_stops = e_line_northbound_stops[e_line_northbound_stops['CURRENT_NE'] == 'IN SERVICE'] ## eliminating 3X stops


In [5]:
e_line_full_segment = all_kcm_segment_data[all_kcm_segment_data['ROUTE_NUM'] == 675] ##675 is the E line's route number

In [6]:
#e_line_northbound_stops.explore()

### Ridership data stuffs

In [7]:
### This cell calculates the departing load from every stop!
departing_load = 0

for stop_num in ridership_data.keys():    
    current_boardings = ridership_data[stop_num][0]
    current_alightings = ridership_data[stop_num][1]

    departing_load = departing_load + current_boardings - current_alightings

    ridership_data[stop_num].append(departing_load)
    #print (f"stop_num - {stop_num}, current_boardings - {current_boardings}, current_alightings - {current_alightings}, departing_load - {departing_load}")

### Now its time to try and draw a line between each stop

In [8]:
### make a function enabling the #shortest_line function to work with just two points.

def shortest_line_between(geom1, geom2):
    return gpd.GeoSeries([geom1]).shortest_line(gpd.GeoSeries([geom2])).iloc[0]


In [9]:
### generate a list of stops 
stop_geometry_dict = {}
for i in range(len(ridership_data.keys())):
    stop_geometry_dict[f"stop_number_{i+1}_geo"] = e_line_northbound_stops.iloc[i]['geometry']

In [10]:
#segment_dictionary

num_stops = len(stop_geometry_dict)

segment_dict = {}

for j in range(0, num_stops-1):
    segment_dict[f"segment_{j+1}"] = shortest_line_between(stop_geometry_dict[f"stop_number_{j+1}_geo"], stop_geometry_dict[f"stop_number_{j+2}_geo"])

In [11]:
ridership_data[7770][2]

18

In [12]:
num_segments = len(segment_dict)

segment_gdf = gpd.GeoDataFrame(
                                {'starting_stop_id': e_line_northbound_stop_list[:num_segments],
                                'ridership_data_dict':[ridership_data[stop_id] for stop_id in e_line_northbound_stop_list[:num_segments]],
                                'geometry': [segment_dict[f"segment_{id + 1}"] for id in range(num_segments)]},
                                geometry = 'geometry', crs = "EPSG:2926"
                              )


In [13]:
segment_gdf

,starting_stop_id,ridership_data_dict,geometry
0,7690,"[12, 0, 12]","LINESTRING (1268255 253055, 1268277 254342.3)"
1,7710,"[14, 6, 20]","LINESTRING (1268277 254342.3, 1268298 255739.1)"
2,7730,"[13, 2, 31]","LINESTRING (1268298 255739.1, 1268313 257094.7)"
3,7750,"[5, 20, 16]","LINESTRING (1268313 257094.7, 1268312 258317.8)"
4,7770,"[8, 6, 18]","LINESTRING (1268312 258317.8, 1268343 259603.1)"
5,7790,"[10, 14, 14]","LINESTRING (1268343 259603.1, 1268358 260977.2)"
6,7810,"[3, 6, 11]","LINESTRING (1268358 260977.2, 1268366 263636)"
7,7850,"[2, 7, 6]","LINESTRING (1268366 263636, 1268386 266091.6)"


In [14]:
avg_ons_list = []
avg_offs_list = []
avg_load_list = []

for i in range(len(segment_gdf)):
    value_trio = segment_gdf['ridership_data_dict'][i]
    avg_ons_list.append(value_trio[0])
    avg_offs_list.append(value_trio[1])
    avg_load_list.append(value_trio[2])




In [15]:
segment_gdf["boardings"] = avg_ons_list
segment_gdf["alightings"] = avg_offs_list
segment_gdf["departing_load"] = avg_load_list

In [16]:
segment_gdf

,starting_stop_id,ridership_data_dict,geometry,boardings,alightings,departing_load
0,7690,"[12, 0, 12]","LINESTRING (1268255 253055, 1268277 254342.3)",12,0,12
1,7710,"[14, 6, 20]","LINESTRING (1268277 254342.3, 1268298 255739.1)",14,6,20
2,7730,"[13, 2, 31]","LINESTRING (1268298 255739.1, 1268313 257094.7)",13,2,31
3,7750,"[5, 20, 16]","LINESTRING (1268313 257094.7, 1268312 258317.8)",5,20,16
4,7770,"[8, 6, 18]","LINESTRING (1268312 258317.8, 1268343 259603.1)",8,6,18
5,7790,"[10, 14, 14]","LINESTRING (1268343 259603.1, 1268358 260977.2)",10,14,14
6,7810,"[3, 6, 11]","LINESTRING (1268358 260977.2, 1268366 263636)",3,6,11
7,7850,"[2, 7, 6]","LINESTRING (1268366 263636, 1268386 266091.6)",2,7,6


### Ok Now the GDF is sorta done..... lets make a map!!

In [17]:
def color_function(feature):
    departing_load = features["properties"]["departing_load"]
    if (departing_load > 10):
        return "#ff0000"
    else:
        return "#008000"

In [18]:
    step = cm.StepColormap(
        ["#d73027", "#fc8d59", "#fee08b", "#d9ef8b", "#91cf60", "#1a9850",], vmin=3, vmax=31, index=[3, 8, 12, 15, 18, 25, 31], caption="step"
    )

    step

In [19]:
def style_function_simple(features):
    departing_load = features["properties"]["departing_load"]
   
    #print (features["properties"]["departing_load"])
    return {
        'weight': departing_load,
        'color': step(departing_load),
        'lineCap': 'butt', # options -> 'round', 'square', 'butt'
        'lineJoin': 'round' #options -> 'round', 'bevel', 'miter'
    }

In [20]:
tt = folium.GeoJsonTooltip(
    fields = ['boardings', 'alightings', 'departing_load']
)

In [21]:
folium_map = folium.Map(location=[47.699389, -122.344733], zoom_start=14, tiles="cartodb positron")


folium.GeoJson(
    data = segment_gdf,
    style_function = style_function_simple,
    tooltip = tt
).add_to(folium_map)

In [22]:
#folium_map

#### Ok now that the single line map is done...... it is time to ***attempt*** to make the boarding/alighting lines

In [23]:
###get the point data frame

e_line_northbound_stops['geometry']

260      POINT (1268255 253055)
262    POINT (1268277 254342.3)
266    POINT (1268298 255739.1)
268    POINT (1268313 257094.7)
295    POINT (1268312 258317.8)
297    POINT (1268343 259603.1)
299    POINT (1268358 260977.2)
303      POINT (1268366 263636)
306    POINT (1268386 266091.6)
Name: geometry, dtype: geometry

In [24]:
#### use shapely.transform to generate new points?

In [25]:
test_point = e_line_northbound_stops['geometry'].iloc[0]

shifted_point = shapely.transform(geometry=test_point, transformation=lambda x,y: (x + 5, y + 5), interleaved=False)

In [26]:
shifted_stops_dict = {}
counter = 0


for point in e_line_northbound_stops['geometry']:
    counter = counter + 1
    shifted_stops_dict[f"stop_{counter}_shifted_point"] = shapely.transform(geometry=point, transformation=lambda x,y: (x + 500, y + 500), interleaved=False)


In [27]:
shifted_points_list = list(shifted_stops_dict.values())

In [28]:
shifted_stops_gdf = gpd.GeoDataFrame(geometry = shifted_points_list, crs = "EPSG:2926")

In [29]:
shift_map = shifted_stops_gdf.explore()

In [30]:
e_line_northbound_stops.explore(m = shift_map)